<a href="https://colab.research.google.com/github/mohmiim/MLIntroduction/blob/master/session-6/MultiVarMultiStep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x
import tensorflow as tf

In [0]:
import os

!wget --no-check-certificate \
    https://raw.githubusercontent.com/mohmiim/MLIntroduction/master/session-6/data/combined_csv.csv \
    -O /tmp/weather.csv

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import csv
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Conv1D,Dense,MaxPooling1D,Flatten, Conv1D, Dropout,TimeDistributed
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import Huber
from math import sqrt

In [0]:
df = pd.read_csv("/tmp/weather.csv")


columnsToConsider = ['p (mbar)', 'T (degC)', 'rho (g/m**3)']
features = df[columnsToConsider]
features.index = df['Date Time']


features.plot(subplots=True)
plt.show()

TRAIN_DATA_SIZE = 260000

dataset = features.values

data_mean = dataset[:TRAIN_DATA_SIZE].mean(axis=0)
data_std = dataset[:TRAIN_DATA_SIZE].std(axis=0)

dataset = (dataset-data_mean)/data_std

print(len(dataset))

training_data = dataset[:TRAIN_DATA_SIZE]
validation_data = dataset[TRAIN_DATA_SIZE:]

print(len(training_data))
print(len(validation_data))

print(data_mean)
print(data_std)


In [0]:
def sequenceData(dataset, target, start_index, end_index, history_size,target_size, step, single_step=False):
    X = []
    y = []
    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size
    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        X.append(dataset[indices])
        if single_step:
            y.append(target[i+target_size])
        else:
            y.append(target[i:i+target_size])
    return np.array(X), np.array(y)


In [0]:
LOOK_AHEAD = 72
STEP = 6
WINDOW_SIZE = 720
BATCH_SIZE = 200
BUFFER_SIZE = 1000

In [0]:
X_train, y_train = sequenceData(dataset, dataset[:, 1], 0,
                                 TRAIN_DATA_SIZE, WINDOW_SIZE,
                                 LOOK_AHEAD, STEP)

X_val, y_val = sequenceData(dataset, dataset[:, 1],
                            TRAIN_DATA_SIZE, None, WINDOW_SIZE,
                            LOOK_AHEAD, STEP)

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

In [0]:
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_data = train_data.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

val_data = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_data = val_data.batch(BATCH_SIZE)

In [0]:
def create_time_steps(length):
    return list(range(-length, 0))

In [0]:
def multi_step_plot(history, true_future, prediction):
    plt.figure(figsize=(12, 6))
    num_in = create_time_steps(len(history))
    num_out = len(true_future)
    plt.plot(num_in, np.array(history[:, 1]), label='History')
    plt.plot(np.arange(num_out)/STEP, np.array(true_future), 'bo',
             label='True Future')
    if prediction.any():
        plt.plot(np.arange(num_out)/STEP, np.array(prediction), 'ro',
                 label='Predicted Future')
        plt.legend(loc='upper left')
    plt.show()

In [0]:
for x, y in train_data.take(1):
    multi_step_plot(x[0], y[0], np.array([0]))

In [0]:
def createConvLSTMModel() :
  model = Sequential()
  model.add(Conv1D(512, 3, activation='relu', input_shape=X_train.shape[-2:]))
  model.add(MaxPooling1D())
  model.add(Conv1D(512, 3, activation='relu'))
  model.add(MaxPooling1D())
  model.add(LSTM(32,return_sequences=True,activation='relu'))
  model.add(LSTM(16, activation='relu'))
  model.add(Dense(72))
  model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mae')
  return model

In [0]:
def createLSTMModel() :
  model = Sequential()
  model.add(LSTM(32,return_sequences=True,activation='relu',input_shape=X_train.shape[-2:]))
  model.add(LSTM(16, activation='relu'))
  model.add(Dense(72))
  model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mae')
  return model

In [163]:
model  = createConvLSTMModel()
model.summary()


Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_33 (Conv1D)           (None, 118, 512)          5120      
_________________________________________________________________
max_pooling1d_31 (MaxPooling (None, 59, 512)           0         
_________________________________________________________________
conv1d_34 (Conv1D)           (None, 57, 512)           786944    
_________________________________________________________________
max_pooling1d_32 (MaxPooling (None, 28, 512)           0         
_________________________________________________________________
lstm_43 (LSTM)               (None, 28, 32)            69760     
_________________________________________________________________
lstm_44 (LSTM)               (None, 16)                3136      
_________________________________________________________________
dense_21 (Dense)             (None, 72)              

In [0]:
EVALUATION_INTERVAL = 200
EPOCHS = 10

history = model.fit(train_data, epochs=EPOCHS)
'''
single_step_history = model.fit(train_data, epochs=EPOCHS,
                                steps_per_epoch=EVALUATION_INTERVAL,
                                validation_data=val_data,
                                validation_steps=50)
'''

Train for 1297 steps
Epoch 1/10
1297/1297 [==============================] - 101s 78ms/step - loss: 0.4125
Epoch 2/10
1181/1297 [==========================>...] - ETA: 8s - loss: 0.2897

In [0]:
for X, y in val_data.take(4):
  multi_step_plot(x[0], y[0], model.predict(X)[0])